In [6]:
import json, sys, os, requests
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [255]:
colors = json.loads(
    requests.get(
        "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/colors/eco-colors.json"
    ).content
)

In [115]:
def get_ons_data_ts(dataset, series, save=True, api_path="https://api.ons.gov.uk"):
    r = requests.get(
        api_path
        + "/timeseries/"
        + series.lower()
        + "/dataset/"
        + dataset.lower()
        + "/data"
    )
    content = json.loads(r.content)
    if save:
        print("Saving DATASET", dataset, "SERIES", series, "to local mirror...")
        open("datasets/ons/" + dataset + "/" + series + ".json", "w").write(
            json.dumps(content)
        )
        print("OK!")
    return content

In [566]:
dataset = "lms"
data = {}
series = ["mgsx", "lf24", "kac3"]
series_colors = {"mgsx": "eco-pink", "lf24": "eco-turquiose", "kac3": "eco-yellow"}
series_offset = {"mgsx": 0.8, "lf24": 0.5, "kac3": 1}
series_domain = {"mgsx": [2, 12], "lf24": [65, 80], "kac3": [-2, 6]}
series_domain2 = {"mgsx": [3.5, 5.5], "lf24": [73, 77], "kac3": [-2, 6]}
for serie in series:
    d = get_ons_data_ts(dataset, serie)
    data[serie] = {
        "title": d["description"]["title"].split(")")[0] + ")",
        "data": d["months"],
    }

Saving DATASET lms SERIES mgsx to local mirror...
OK!
Saving DATASET lms SERIES lf24 to local mirror...
OK!
Saving DATASET lms SERIES kac3 to local mirror...
OK!


In [599]:
live = False
layers = []
for serie in series:
    if live:
        l = "_live"
        url = (
            "https://api.allorigins.win/raw?url=https://api.ons.gov.uk/timeseries/"
            + serie
            + "/dataset/"
            + dataset
            + "/data"
        )
    else:
        l = ""
        url = (
            "https://raw.githubusercontent.com/EconomicsObservatory/ECOdataHUB/main/datasets/ons/"
            + dataset
            + "/"
            + serie
            + ".json"
        )
    base = (
        alt.Chart(
            alt.Data(
                url=url,
                format=alt.DataFormat(type="json", property="months"),
            )
        )
        .transform_calculate(serie=repr(serie))
        .transform_calculate(label="datum.year+' '+datum.month")
        .encode(x=alt.X("date:T", axis=alt.Axis(grid=True, title="")))
    )
    line = base.mark_area(
        interpolate="monotone",
        fillOpacity=0.7,
        stroke=colors["eco-gray"],
        strokeWidth=0.5,
        color=alt.Gradient(
            gradient="linear",
            stops=[
                alt.GradientStop(color="white", offset=0),
                alt.GradientStop(
                    color=colors[series_colors[serie]], offset=series_offset[serie]
                ),
            ],
            x1=0.8,
            x2=1,
            y1=1,
            y2=0,
        ),
    ).encode(
        y=alt.Y(
            "value:Q",
            axis=alt.Axis(grid=True, title=""),
        )
    )
    # Create a selection that chooses the nearest point & selects based on x-value
    nearest = alt.selection(
        type="single", nearest=True, on="mouseover", fields=["date"], empty="none"
    )
    # Transparent selectors across the chart. This is what tells us
    # the x-value of the cursor
    selectors = (
        base.mark_point()
        .encode(
            opacity=alt.value(0),
        )
        .add_selection(nearest)
    )

    # Draw points on the line, and highlight based on selection
    points = line.mark_point().encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0))
    )

    # Draw text labels near the points, and highlight based on selection
    text = line.mark_text(align="left", dx=-25, dy=35).encode(
        text=alt.condition(nearest, "value:N", alt.value(" "))
    )

    # Draw text labels near the points, and highlight based on selection
    text2 = line.mark_text(align="right", dx=5, dy=10,).encode(
        text=alt.condition(nearest, "label:N", alt.value(" ")),
        x=alt.value(390),
        y=alt.value(0),
    )

    # Draw text labels near the points, and highlight based on selection
    text3 = line.mark_text(align="right", dx=5, dy=10, fontSize=13).encode(
        text=alt.condition("year(datum.date)>=2021", "value:Q", alt.value(" ")),
        x=alt.value(92),
        y=alt.value(3),
    )

    # Draw a rule at the location of the selection
    rules = base.mark_rule(color=colors["eco-gray"]).transform_filter(nearest)

    # Put the five layers into a chart and bind the data
    layer1 = alt.layer(
        line.encode(
            y=alt.Y(
                "value:Q",
                # scale=alt.Scale(domain=series_domain[serie]),
            )
        ),
        selectors,
        points,
        rules,
        text,
        text2,
    ).properties(width=400, height=150, title=data[serie]["title"])
    layer2 = (
        alt.layer(
            line.encode(
                y=alt.Y(
                    "value:Q",
                    # scale=alt.Scale(domain=series_domain2[serie]),
                )
            ),
            selectors,
            points,
            rules,
            text,
            text3,
        )
        .properties(width=100, height=150)
        .transform_filter("year(datum.date)>=2020")
    )
    layers.append(alt.hconcat(layer1, layer2, spacing=20))

layers = alt.vconcat(layers[0], layers[1], layers[2]).configure_view(stroke=None)
layers.save("panels/" + dataset + l + ".json")
layers

alt.VConcatChart(...)

In [595]:
live = False
layers = []
for serie in series:
    if live:
        l = "_live"
        url = (
            "https://api.allorigins.win/raw?url=https://api.ons.gov.uk/timeseries/"
            + serie
            + "/dataset/"
            + dataset
            + "/data"
        )
    else:
        l = ""
        url = (
            "https://raw.githubusercontent.com/EconomicsObservatory/ECOdataHUB/main/datasets/ons/"
            + dataset
            + "/"
            + serie
            + ".json"
        )
    base = (
        alt.Chart(
            alt.Data(
                url=url,
                format=alt.DataFormat(type="json", property="months"),
            )
        )
        .transform_calculate(serie=repr(serie))
        .transform_calculate(label="datum.year+' '+datum.month")
        .encode(
            x=alt.X("date:T", axis=alt.Axis(grid=True, title="")),
            y=alt.Y("value:Q"),
        )
        .mark_text(align="left", dx=-25, dy=35)
    )
    line = base.mark_area(
        interpolate="monotone",
        fillOpacity=0.7,
        stroke=colors["eco-gray"],
        strokeWidth=0.5,
        color=alt.Gradient(
            gradient="linear",
            stops=[
                alt.GradientStop(color="white", offset=0),
                alt.GradientStop(color=colors[series_colors[serie]], offset=1),
            ],
            x1=0.8,
            x2=1,
            y1=1,
            y2=0,
        ),
    )
    # Create a selection that chooses the nearest point & selects based on x-value
    nearest = alt.selection(
        type="single", nearest=True, on="mouseover", fields=["date"], empty="none"
    )
    # Transparent selectors across the chart. This is what tells us
    # the x-value of the cursor
    selectors = (
        base.mark_point()
        .encode(
            opacity=alt.value(0),
        )
        .add_selection(nearest)
    )

    # Draw points on the line, and highlight based on selection
    points = line.mark_point().encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0))
    )

    # Draw text labels near the points, and highlight based on selection
    text = line.mark_text(align="left", dx=-25, dy=35).encode(
        text=alt.condition(nearest, "value:N", alt.value(" "))
    )

    # Draw text labels near the points, and highlight based on selection
    text2 = line.mark_text(align="right", dx=5, dy=10,).encode(
        text=alt.condition(nearest, "label:N", alt.value(" ")),
        x=alt.value(390),
        y=alt.value(0),
    )

    # Draw text labels near the points, and highlight based on selection
    text3 = line.mark_text(align="right", dx=5, dy=10, fontSize=13).encode(
        text=alt.condition("year(datum.date)>=2021", "value:Q", alt.value(" ")),
        x=alt.value(92),
        y=alt.value(3),
    )

    # Draw a rule at the location of the selection
    rules = base.mark_rule(color=colors["eco-gray"]).transform_filter(nearest)

    # Put the five layers into a chart and bind the data
    layer1 = alt.layer(
        line.encode(
            y=alt.Y(
                "value:Q",
                # scale=alt.Scale(domain=series_domain[serie]),
            )
        ),
        selectors,
        points,
        rules,
        text,
        text2,
    ).properties(width=400, height=150)
    layer2 = (
        alt.layer(
            #             line.encode(
            #                 y=alt.Y(
            #                     "value:Q",
            #                     scale=alt.Scale(domain=[0, 10]),
            #                 )
            #             ),
            base,
            selectors,
            points,
            rules,
            text,
            text3,
        )
        .properties(width=100, height=150)
        .transform_filter("year(datum.date)>=2020")
    )
    layers.append(alt.hconcat(layer1, layer2, spacing=20))

layers = alt.vconcat(layers[0], layers[1], layers[2])  # .configure_view(stroke=None)
layers.save("panels/" + dataset + l + ".json")
layers

alt.VConcatChart(...)

In [549]:
base

SchemaValidationError: Invalid specification

        altair.vegalite.v4.api.Chart, validating 'required'

        'mark' is a required property
        

alt.Chart(...)